In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, concat_ws, row_number
from pyspark.sql.functions import to_date, date_format
from pyspark.sql.window import Window

dbutils.widgets.text("Record_Type", "customer", "Record Type Parameter")
record_type = dbutils.widgets.get("Record_Type")

spark = SparkSession.builder.getOrCreate()


In [0]:
# Load previous result
df = spark.table("my_database.my_schema.int_ar_receipt_integrate")

df.show()

In [0]:


# Sort the DataFrame (replace 'colA' with your desired sort column)
sorted_df = df.orderBy("receipt_number")


windowSpec = Window.orderBy("receipt_number")  # or another column (or multiple columns)
df_seq = sorted_df.withColumn("seq_num", row_number().over(windowSpec))

# Add default value column
df_seq = df_seq.withColumn("Record_Type", lit(record_type))

# Add concatenated column (change colA and colB as needed)
df_seq = df_seq.withColumn("Unique_column", concat_ws("_", df_seq.customer_name, df_seq.receipt_number))

# df_seq = df_seq.withColumn(
#     "date",
#     date_format(to_date("date", "yyyy-MM-dd"), "dd-MM-yyyy")
# )

df_seq.show()

In [0]:
# Save as new table
df_seq.write.format("delta").mode("overwrite").saveAsTable("my_database.my_schema.Trn_ar_receipt_transform")